# Q-learning from scratch
The value of a state or a state-action pair is the expected return if we start from that state or state-action pair. Using value function, we can define the Bellman equation and assort to temporal difference learning which is a powerful method in RL. In the sequel, We define the main component in the $Q$-learning algorithm.

Note that in $Q$ learning, we do not directly parameterize the policy. We parametrize the $Q$-function and the policy is defined by maximizing (or minimizing depending on whether you consider reward or cost) the $Q$-function. So, what we learn is the $Q$-function. The policy is obtained from the $Q$ function. 

## 1.  The $Q$- function 
The quality function is equal to the expected return reward for taking an arbitrary action $a$ and then following the policy $\pi$. The following equation defines the Bellman equation for the $Q$-function

\begin{align*}
\large
Q(s,a) =r(s,a)+ \gamma\: \mathbf{E}[Q(s^{\prime}, \pi(s^{\prime}))]
\end{align*}
where the policy $\large \pi$ is the action maximizes the expected reward starting in $s$
\begin{align*}
\pi = \arg \max_a Q (s,a).
\end{align*}
and  $r(s,a),\:0 \leq \gamma \leq 1$ are the immediate reward and the discount factor.

> _In a $Q$-learning algorithm, we learn the $Q$ function._

If we prefer to work with cost $c(s,a)=-r(s,a)$, we can replace $r(s,a) $ with $c(s,a)$ in the $Q$ function and define the policy as $\pi = \arg \min_a Q (s,a)$. An important observation is that the equation which defines $Q(s,a)$ is actually a Bellman equation: The quality function of the current state-action pair $(s,a)$ is the immediate reward plus the quality of the next state-action pair $(s^{\prime}, \pi(s^{\prime}))$.

Finding the policy as $\pi = \arg \max_a Q (s,a)$ needs further consideration. To find the policy in each action, we need to solve an optimization problem; i.e. select the action $a$ to maximize $Q$. Since we have two possible scenarios where the action space can be discrete or continuous, we need to define the $Q$ function for each case properly so that it is possible to optimize the $Q$ function without appealing to advanced optimization techniques. From here on, we treat discrete and continuous action spaces differently.


### 1.1 Discrete action space
When there is a finite number of $n_a$ actions, we consider a network which takes the state $s$ as the input and generates $n_a$ outputs. Each output is $Q(s,a)$ for all $a \in \mathcal{A}$ and $Q(s,a)$ is obtained by indexing into the output vector `network(state)`. The policy $\pi$ is the index which the output of the network is maximized.

```
network = keras.Sequential([
        keras.layers.Dense(30, input_dim=n_s, activation='relu'),
        keras.layers.Dense(30, activation='relu'),
        keras.layers.Dense(30, activation='relu'),
        keras.layers.Dense(n_a)])
```
In the above code, we build the network. The network takes a state of dimension $n_s$ as the input and uses it in a fully connected layer with 30 neurons, with the activation function as relu, followed by two layers each with 30 neurons and with the activation function as relu. Then, we have the last layer which has $n_a$ number of outputs. The parameters in the networks are biases and weights in the layers. 
	
Using the network which we just defined, we can define the policy as the argument that maximizes the $Q$ function
<code>policy = np.argmax(network(state))<code>

### 1.2 Continuous action space
When the action space is continuous, we cannot follow the same lines as the discrete action space case because simply we have an infinite number of actions. In this case, the $Q$ function is built by a network which takes the state $s$ and action $a$ as the input and generates a single value $Q(s,a)$ as the output. The policy in each state $s$ is given by $\arg_{a} \max Q(s,a)$. Since we are not interested (neither possible nor making sense) in solving an optimization problem in each state, we select a structure for the $Q$ function such that the optimization problem is carried out analytically. A good example is a linear quadratic problem where it is proved that the $Q$ function is quadratic in $s$ and $a$

\begin{align*}
Q(s,a) =\begin{bmatrix}
s^{\dagger} & a^{\dagger}
\end{bmatrix}\begin{bmatrix}
g_{ss} & g_{sa}\\
g_{sa}^{\dagger} & g_{aa} \end{bmatrix}\begin{bmatrix}
s\\a \end{bmatrix}= z^{\dagger} G z
\end{align*}

where $z=\begin{bmatrix}
s^{\dagger} & a^{\dagger}
\end{bmatrix}^{\dagger} $ and $G= \begin{bmatrix}
g_{ss} & g_{sa}\\
g_{sa}^{\dagger} & g_{aa} \end{bmatrix}$.

The policy $\pi$ is obtained by mathematical maximization of the function $Q(s,a)$
\begin{align*}
\pi = -g_{aa}^{-1}g_{sa}^{\dagger} \: s.
\end{align*}

## 2. Temporal difference learning
As the name implies, in a $Q$-learning algorithm, we build a (possibly deep) network and learn the $Q$-function.
If this `network` represents the __true__ $Q$-function, then it satisfies the Bellman equation in section 1. Before learning, however, the network does not represent the true $Q$ function. We define the _temporal difference error $e$_

\begin{align*}
 e = r(s,a)+ \gamma\: \mathbf{E}[Q(s^{\prime}, \pi(s^{\prime}))]-Q(s,a)
\end{align*}
We learn the parameters in the network $Q$ to minimize the mean square error (mse) $\dfrac{1}{2} \: e^2$. In the sequel, we show how to minimize $\dfrac{1}{2} \: e^2$ in discrete and continuous action space cases.

### 2.1 Discrete action space
To learn the parameters in discrete action space case, we define an mse cost for the network

<code>network.compile(loss='mean_squared_error')<code>

After configuring the network, the last step is to feed the network with `states`, `actions`, `next_states`, and `dones` and update the parameters of the network. Note that `dones` is an array of Booleans with the same length as `states`. The ith element in `dones` is `True` if the ith state in `states` is the last state in the episode (showing that the episode eded) and `False` otherwise.
```
eps_length = len(states)
states = np.vstack(states)
q_target = network(states).numpy
for i in range(eps_length):
    if dones[i]:
        q_target[i, actions[i]] = rewards[i]
    else:
        q_target[i, actions[i]] = rewards[i] + Gamma * tf.math.reduce_max(network(next_states[i])).numpy()
        
loss = network.train_on_batch(states, q_target)
```

### 2.2 Continuous action space
For a quadratic $Q=z^{\dagger} G z$ function, the matrix $G$ is learned by _Least Square Temporal Difference learning (LSTD)_. For the vector $v =[v_1,\:v_2,...,v_n]^{\dagger}$, define the operator $vecv(v)=[v_1^2,\: 2v_1 v_2,...,2v_1 v_n,\:v_2^2,...,\: 2v_2 v_n,\: ... ,v_n^2]^{\dagger}$. The LSTD estimation of the symmetric matrix $G$ is given by

\begin{align*}		
vecs(G)= ( \frac{1}{T}\sum_{t=1}^{T} \Psi_t (\Psi_t-\gamma \Psi_{t+1})^{\dagger} )^{-1} ( \frac{1}{T}\sum_{t=1}^{T} \Psi_t r_t ), 
\end{align*}
where
* $\Psi_t=vecv(z_t),\:z_t=\begin{bmatrix}s_t^{\dagger} & a_t^{\dagger}\end{bmatrix}^{\dagger}$,
* $T$ is the rollout length.


## 3. How to select action $a$? _exploration vs. exploitation_
You have probably heard about _exploration vs. exploitation_. This concept is best described by this example. Suppose that you want to go to a restaurant in town. Exploration means that you select a random restaurant that you have not tried before. Exploitation means that you go to your favorite one. The good point with exploitation is that you like what you'll eat and the good point with exploration is that you might find something that you like more than your favorite. 

The same thing happens in RL. If the agent only sticks to exploitation, it can never improve its policy and it will get stuck in a local optimum forever. On the other hand, if the agent only explores, it never uses what it has learned and only tries random things. It is important to balance the levels of exploration and exploitation. The simplest way of selecting $a$ to have both exploration and exploitation is described here for discrete and continuous action space.

### 3.1 Discrete action space
When there is a finite number of actions, the action $a$ is selected as follows. We set a level $0<\epsilon<1$ (for example $\epsilon = 0.1$) and we select a random number $r\sim [0,\:1]$. If $r<\epsilon$, we explore by selecting a random action otherwise, we follow the policy by maximizing the $Q$ function 

\begin{align*}
a = \begin{cases}
\text{random action}\quad \text{if   } r \:<\: \epsilon\\
\arg \max_a Q (s,a) \quad \text{Otherwise}
\end{cases}
\end{align*}

The following code implement the above expression:

```
if np.random.random() <= epsilon:
    selected_action = env.action_space.sample()
else:
    selected_action = np.argmax(network(state))
```


### 3.2 Continuous action space
When the action space is continuous, the action $a$ is selected as the optimal policy plus some randomness. Let $r \sim \mathcal{N}(0,\sigma^2)$

\begin{align*}
a = \arg \max_a Q (s,a) + r
\end{align*}

For example, in the linear quadratic case, the above expression can be coded as the following where $n_a$ is the dimension of the input

```a = -g_aa^{-1} @ g_sa.T @ state + stddev * np.random.randn(n_a)```

Note that smaller stddev, less exploration. (The symbol @ represent matrix multiplication.)

## 4. Putting all together
Now, we put all steps together to run $Q$-learning algorithm. 

First, we build a (deep) network to represent $Q(s,a)$. See section 1. Then, we iteratively improve the network. In each iteration of the algorithm, we do the following
* i. We rollout the environment to collect data for $Q$-learning by following these steps:
    * i.a. We initialize empty histories for `states=[]`, `actions=[]`, `rewards=[]`, `next_states=[]`, `dones[]`.
    * i.b. We observe the `state` $s$ and select the `action` $a$. See section 3.
    * i.c. We derive the environment using $a$ and observe the `reward` $r$, the `next_state` $s^{\prime}$ and the Boolean $done$ (which is `True` if the episode has ended and `False` otherwise).
    * i.d. We add $s,\:a,\:r,\:s^{\prime},\:done$ to the history batch `states`, `actions`, `rewards`, `next_states`, `dones`.
    * i.e. We continue from i.b. until the episode ends.
* ii. We supply `states`, `actions`, `rewards`, `next_states` to the network and optimize the parameters of the network. See section 2.

## 5. Related files
* [$Q$ learning for discrete action space: The cartpole example (study and code)](q_on_cartpole_notebook.ipynb)
* [$Q$ learning for discrete action space: The cartpole example (only code)](./cartpole/q_on_cartpole.py)
* [$Q$ learning for continuous action space: The linear quadratic (study and code)](q_on_lq_notebook.ipynb)
* [$Q$ learning for continuous action space: The linear quadratic (only code)](./lq/q_on_lq.py)